This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [5]:
import pandas as pd
import os
import geocoder
import requests

In [6]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    count_1 = df['Offense 1'].value_counts()
    count_2 = df['Offense 2'].value_counts()
    count_3 = df['Offense 3'].value_counts()
    count_4 = df['Offense 4'].value_counts()
    count_5 = df['Offense 5'].value_counts()

    counts = [count_1, count_2, count_3, count_4, count_5]
    for count in counts:
        if count.empty:
            df = df.drop(count.name, axis = 1)
            
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

NotADirectoryError: [WinError 267] The directory name is invalid: './raw/aprilComplete.csv'

In [7]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [9]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4030 N TEUTONIA AV, Milwaukee, WI - [43.090431652877115, -87.93917835376055]
Parsed: 4914 N 37TH ST, Milwaukee, WI - [43.106786413266775, -87.95894535356932]
Parsed: 4644 N 31ST ST, Milwaukee, WI - [43.1018719103526, -87.95167343738835]
Parsed: 5358 N 39TH ST, Milwaukee, WI - [43.11503882653355, -87.9611443607827]
Parsed: 5446 N 38TH ST, Milwaukee, WI - [43.116632968636395, -87.95990236078269]
Parsed: 3001 W SILVER SPRING DR, Milwaukee, WI - [43.1191045650645, -87.95106163474526]
Parsed: 3806 W DOUGLAS AV, Milwaukee, WI - [43.13126048211083, -87.95973180391252]
Parsed: 4242 N 17TH ST, Milwaukee, WI - [43.09478966472389, -87.93185589353128]
Parsed: 6883 N DARIEN ST, Milwaukee, WI - [43.14286722766769, -87.95721719123706]
Parsed: 5175 N TEUTONIA AV, Milwaukee, WI - [43.111179978009396, -87.94995413358069]
Parsed: 4628 W STARK ST, Milwaukee, WI - [43.1064725115412, -87.971211]
Parsed: 3100 W CAMERON AV, Milwaukee, WI - [43.10732046047074, -87.95182033236193]
Parsed: 3807 W CHEYENN

Parsed: 3136 N 55TH ST, Milwaukee, WI - [43.07617872300767, -87.98195236078269]
Parsed: 3090 N 53RD ST, Milwaukee, WI - [43.075106664723876, -87.97965292296163]
Parsed: 3029 N 49TH ST, Milwaukee, WI - [43.07399609177099, -87.97485332930152]
Parsed: 5000 W CHAMBERS ST, Milwaukee, WI - [43.073474483841586, -87.9761812590745]
Parsed: 1202 N 44TH ST, Milwaukee, WI - [43.04607338773155, -87.96913644460173]
Parsed: 10423 W BLUE MOUND RD, Milwaukee, WI - [43.034612356079414, -88.04251398247295]
Parsed: 4513 W MARTIN DR, Milwaukee, WI - [43.04415161223979, -87.97044748765074]
Parsed: 6657 W CHAMBERS ST, Milwaukee, WI - [43.07352151399405, -87.9956235]
Parsed: 3006 N 73RD ST, Milwaukee, WI - [43.073757, -88.0035039190665]
Parsed: 6860 W APPLETON AV, Milwaukee, WI - [43.08273039745507, -87.99816468218036]
Parsed: 9223 W MT VERNON AV, Milwaukee, WI - [43.034011513994045, -88.0282715770097]
Parsed: 2658 N 54TH ST, Milwaukee, WI - [43.06740338773156, -87.98095591185313]
Parsed: 5100 W CENTER ST, Mi

Parsed: 1508 W ORCHARD ST, Milwaukee, WI - [43.01603945325738, -87.93108947155058]
Parsed: 1128 S 10TH ST, Milwaukee, WI - [43.019460335276136, -87.92392291185313]
Parsed: 911 W NATIONAL AV, Milwaukee, WI - [43.02321551731225, -87.9228225]
Parsed: 500 W NATIONAL AV, Milwaukee, WI - [43.02331866971551, -87.91693764803088]
Parsed: 1573 S 9TH ST, Milwaukee, WI - [43.0143927799065, -87.92265754818492]
Parsed: 2173 S 6TH ST #LWR, Milwaukee, WI - [43.004853, -87.91879161368207]
Parsed: 2038 W GREENFIELD AV, Milwaukee, WI - [43.0170465004327, -87.93912166763806]
Parsed: 2000 W NATIONAL AV, Milwaukee, WI - [43.02278449711447, -87.93830583680534]
Parsed: 1313 S 6TH ST, Milwaukee, WI - [43.01756096863639, -87.9182775337582]
Parsed: 1112 W GRANT ST, Milwaukee, WI - [43.00469860975839, -87.92603468075862]
Parsed: 2150 S 18TH ST, Milwaukee, WI - [43.005427658895485, -87.93536400678066]
Parsed: 620 W GREENFIELD AV, Milwaukee, WI - [43.01712139089458, -87.91866349141551]
Parsed: 1820 W BURNHAM ST, Mi

Parsed: 3348 S 12TH ST, Milwaukee, WI - [42.98355824562873, -87.9275739884588]
Parsed: 2636 S LOGAN AV, Milwaukee, WI - [42.99636541326677, -87.89671493738835]
Parsed: 2115 S 1ST ST, Milwaukee, WI - [43.005880645017044, -87.91127805770596]
Parsed: 2400 S LOGAN AV, Milwaukee, WI - [43.0004967485429, -87.89667340074465]
Parsed: 600 W HOLT AV, Milwaukee, WI - [42.98177563945035, -87.91866809714044]
Parsed: 2276 S CHASE AV, Milwaukee, WI - [43.00301052484274, -87.91833001912993]
Parsed: 328 E ROSEDALE AV, Milwaukee, WI - [42.995225460470756, -87.90592066763806]
Parsed: 2726 S 15TH ST, Milwaukee, WI - [42.99496272300769, -87.93130494460172]
Parsed: 515 E OTJEN ST, Milwaukee, WI - [42.99776706797868, -87.90319251067581]
Parsed: 3323 S 6TH ST, Milwaukee, WI - [42.98415625867046, -87.9192080809335]
Parsed: 2429 S AUSTIN ST, Milwaukee, WI - [43.000351239800324, -87.9069715226497]
Parsed: 915 E IDAHO ST, Milwaukee, WI - [42.99097619114655, -87.89783598431548]
Parsed: 600 W MORGAN AV, Milwaukee, 

Parsed: 2403 N 20TH ST, Milwaukee, WI - [43.062434025535225, -87.93762110646873]
Parsed: 963 N 33RD ST, Milwaukee, WI - [43.04265890765839, -87.95527984805685]
Parsed: 2700 W NORTH AV, Milwaukee, WI - [43.060720509376814, -87.94813405785018]
Parsed: 2236 N 16TH ST, Milwaukee, WI - [43.05974438773154, -87.93259343738835]
Parsed: 3500 W STATE ST, Milwaukee, WI - [43.04345052407982, -87.95772857790392]
Parsed: 1902 W FOND DU LAC AV, Milwaukee, WI - [43.059074515552794, -87.9367694717522]
Parsed: 2329 N 38TH ST, Milwaukee, WI - [43.061126652130774, -87.96113884786922]
Parsed: 2645 N 24TH PL, Milwaukee, WI - [43.066953167638076, -87.9437186281088]
Parsed: 2415 W FOND DU LAC AV, Milwaukee, WI - [43.06379344627518, -87.94296040663033]
Parsed: 2309 N 50TH ST, Milwaukee, WI - [43.060827825054794, -87.97621687391062]
Parsed: 2146 N 11TH ST, Milwaukee, WI - [43.05889863918864, -87.92548240074464]
Parsed: 2149 N 28TH ST, Milwaukee, WI - [43.05889841909516, -87.94872957372014]
Parsed: 2436 N 17TH S

...

Parsed: 6401 S 13TH ST, Milwaukee, WI - [42.928111878356134, -87.93040502480846]
Parsed: 500 W HOWARD AV, Milwaukee, WI - [42.97379150432783, -87.91711297446477]
Parsed: 4600 S 13TH ST, Milwaukee, WI - [42.96081566472387, -87.92917499509525]
Parsed: 3069 W GREEN AV, Milwaukee, WI - [42.939197692797535, -87.95310614519092]
Parsed: 5300 S HOWELL AV, Milwaukee, WI - [42.94782900000001, -87.90955399624909]
Parsed: 4400 S 13TH ST, Milwaukee, WI - [42.96438955536962, -87.92915449004626]
Parsed: 4100 S 27TH ST, Milwaukee, WI - [42.970275555369625, -87.94834545946208]
Parsed: 5727 S 27TH ST, Milwaukee, WI - [42.94038564363205, -87.94935747331]
Parsed: 4161 S HOWELL AV, Milwaukee, WI - [42.968969025535216, -87.90969101543632]
Parsed: 3138 S 15TH PL, Milwaukee, WI - [42.98734758090484, -87.93263491185313]
Parsed: 4226 S 4TH ST, Milwaukee, WI - [42.96765574854291, -87.91567246292358]
Parsed: 2242 S 17TH ST, Milwaukee, WI - [43.003717742714514, -87.93439343738835]
Parsed: 3105 S 15TH PL, Milwaukee

Parsed: 2113 N 27TH ST, Milwaukee, WI - [43.058196251457105, -87.94755957372014]
Parsed: 2165 N 42ND ST, Milwaukee, WI - [43.0588704446304, -87.9658606392173]
Parsed: 1220 W VLIET ST, Milwaukee, WI - [43.048580464942816, -87.9275335194757]
Parsed: 1846 N 29TH ST, Milwaukee, WI - [43.054784748542914, -87.94988243738835]
Parsed: 1703 N 33RD ST, Milwaukee, WI - [43.05297683819032, -87.95530859536022]
Parsed: 3500 W WELLS ST, Milwaukee, WI - [43.040374257761144, -87.95777908134663]
Parsed: 1636 N 14TH ST, Milwaukee, WI - [43.05212071717929, -87.92998294070658]
Parsed: 1718 W GALENA ST, Milwaukee, WI - [43.051376460470756, -87.93468]
Parsed: 810 W HADLEY ST, Milwaukee, WI - [43.06945650764606, -87.92192233236193]
Parsed: 2434 N 28TH ST, Milwaukee, WI - [43.0630467219092, -87.9486382349541]
Parsed: 1301 W CHERRY ST, Milwaukee, WI - [43.04993048124546, -87.92876266763805]
Parsed: 2233 N 16TH ST, Milwaukee, WI - [43.059584710204575, -87.9326976197315]
Parsed: 3211 W WALNUT ST, Milwaukee, WI - 

Parsed: 901 E CENTER ST, Milwaukee, WI - [43.06744854674261, -87.90018227699233]
Parsed: 2817 N BOOTH ST, Milwaukee, WI - [43.06975317346644, -87.90410060646872]
Parsed: 1030 E NORTH AV, Milwaukee, WI - [43.06020950432784, -87.8983422485429]
Parsed: 3234 N FRATNEY ST, Milwaukee, WI - [43.077329245628704, -87.90142540074464]
Parsed: 1800 E LINNWOOD AV, Milwaukee, WI - [43.07290822495793, -87.88782427504208]
Parsed: 1640 N VAN BUREN ST, Milwaukee, WI - [43.05167346887818, -87.90334228014885]
Parsed: 1900 N WARREN AV, Milwaukee, WI - [43.056117, -87.89367542627986]
Parsed: 2903 N MURRAY AV #14, Milwaukee, WI - [43.07126400000001, -87.8854701503258]
Parsed: 2530 N FARWELL AV, Milwaukee, WI - [43.064445401677496, -87.88178076926863]
Parsed: 2951 N DOWNER AV, Milwaukee, WI - [43.07211016763807, -87.87803007372014]
Parsed: 1218 E KANE PL, Milwaukee, WI - [43.056105504327824, -87.89567047446477]
Parsed: 818 E CENTER ST, Milwaukee, WI - [43.067505518754565, -87.90084016763807]
Parsed: 3006 N PI

Parsed: 208 E CAPITOL DR, Milwaukee, WI - [43.08936546047074, -87.90888213918865]
Parsed: 2961 N 5TH ST, Milwaukee, WI - [43.072541832361935, -87.916969092042]
Parsed: 2948 N DR MARTIN LUTHER KING DR, Milwaukee, WI - [43.07206580301548, -87.91395415224109]
Parsed: 1100 W LOCUST ST, Milwaukee, WI - [43.07136022298921, -87.9254027229892]
Parsed: 3557 N 10TH ST, Milwaukee, WI - [43.08288392200936, -87.92356909925536]
Parsed: 3435 N 10TH ST, Milwaukee, WI - [43.08072903868715, -87.92364465188965]
Parsed: 225 E CAPITOL DR, Milwaukee, WI - [43.089198520630475, -87.90848322202673]
Parsed: 3527 N 9TH ST, Milwaukee, WI - [43.082253754371294, -87.92234059925535]
Parsed: 3559 N 11TH ST, Milwaukee, WI - [43.08292883819033, -87.92478762479058]
Parsed: 2822 N 1ST ST #207, Milwaukee, WI - [43.07004145793194, -87.9109929190665]
Parsed: 2105 N BOOTH ST, Milwaukee, WI - [43.05792742492355, -87.90414801154118]
Parsed: 380 E CAPITOL DR, Milwaukee, WI - [43.08934048773672, -87.90600143782105]
Parsed: 3221 

Parsed: 3124 N 27TH ST, Milwaukee, WI - [43.07583697446478, -87.94720238631791]
Parsed: 4631 W RICE WA, Milwaukee, WI - [43.09662325513756, -87.96827507873967]
Parsed: 2400 W BURLEIGH ST, Milwaukee, WI - [43.07517449321933, -87.94250147155059]
Parsed: 3153 N 47TH ST, Milwaukee, WI - [43.0764204446304, -87.97263913200395]
Parsed: 3830 N 28TH ST, Milwaukee, WI - [43.0868964970858, -87.94825538631792]
Parsed: 4310 W CONGRESS ST, Milwaukee, WI - [43.09741552596792, -87.96758358090484]
Parsed: 4284 N SERCOMBE RD #LWR, Milwaukee, WI - [43.09505353675929, -87.96032840339903]
Parsed: 2207 S MUSKEGO AV, Milwaukee, WI - [43.00461333166945, -87.9433347140053]
Parsed: 3134 W BURNHAM ST, Milwaukee, WI - [43.010353486005954, -87.95378266763807]
Parsed: 3434 W OKLAHOMA AV, Milwaukee, WI - [42.98850246047076, -87.95767091909515]
Parsed: 1750 S MUSKEGO AV, Milwaukee, WI - [43.011479630850125, -87.93874806437198]
Parsed: 2230 S 29TH ST, Milwaukee, WI - [43.00416785789716, -87.95043241185313]
Parsed: 906